# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

In [4]:
#
a = "This is a " +\
    "string"

In [6]:
#
a = ("This is a " +
    "string")

In [8]:
a =  """"
This is one line.
This is another
"""

In [9]:
a

'"\nThis is one line.\nThis is another\n'

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [15]:
thing = pd.io.sql.read_sql(
'''
SELECT *                  ---* means everything    
    FROM properties       --- from a table, properties is a table 
    LIMIT 10              --- limit the size of data it shows
;
''', con = db)          # con means the connection


In [17]:
type(thing)          # means it's a dataframe

pandas.core.frame.DataFrame

In [18]:
thing.head(2)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,2121978635,"829 Folsom St UNIT 906, San Francisco, CA",37781429,-122401860,0,...,557.0,sqft,NaN,None,2010.0
1,89239580,"690 Market St UNIT 1705, San Francisco, CA",37788246,-122403198,0,...,1050.0,sqft,NaN,None,2007.0


In [19]:
thing = pd.io.sql.read_sql(
'''
SELECT id, size           --- here it only gets columns: id and size
    FROM properties       --- from a table, properties is a table 
    LIMIT 10              --- limit the size of data it shows
;
''', con = db)          # con means the connection


In [20]:
thing

,id,size
0,2121978635,557.0
1,89239580,1050.0
2,15131782,937.0
3,15179502,1574.0
4,52266124,1205.0
5,2100994004,400.0
6,15067755,541.0
7,15112556,1175.0
8,15133321,3094.0
9,61288341,1405.0


In [21]:
thing = pd.io.sql.read_sql(
'''
SELECT id, size AS size_foo_bar                 --- you could rename the columns to your like
    FROM properties       --- from a table, properties is a table 
    LIMIT 10              --- limit the size of data it shows
;
''', con = db)          # con means the connection
thing       
# it is conventional that we put SQL statements in uppercase, and everything else in lowercase

,id,size_foo_bar
0,2121978635,557.0
1,89239580,1050.0
2,15131782,937.0
3,15179502,1574.0
4,52266124,1205.0
5,2100994004,400.0
6,15067755,541.0
7,15112556,1175.0
8,15133321,3094.0
9,61288341,1405.0


In [ ]:
thing = pd.io.sql.read_sql(
'''
SELECT properties.id, properties.size AS size_foo_bar                ---specify which table to pull data
    FROM properties       --- from a table, properties is a table 
    LIMIT 10              --- limit the size of data it shows
;
''', con = db)          # con means the connection


### `SELECT`

In [22]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [23]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'      --- specify conditions for it 
;
''', con = db)

,id,sale_price
0,80749447,1.15
1,15143887,2.10
2,15117639,1.35
3,80743040,1.48
4,15134909,1.09
...,...,...
570,2101028916,2.45
571,15091950,1.75
572,89237766,1.16
573,15197693,1.36


In [24]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price/1000000 as sale_price
    FROM transactions
    WHERE sale_price_unit = '$'      --- specify conditions for it 
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [ ]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [25]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)
# same thing we did in the last cell but in one single command

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `WITH`

In [26]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')
    
    --- with created a table in memory, not an actual table in the database
    
SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `JOIN`

In [27]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id                 ---specify how it should be linked
;
''', con = db)



# difference between union and join 
# union is going to stack one on the other
# join is going to link the two together by the link you specify

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [28]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [29]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

,count
0,164
1,142
2,316
3,226
4,103
5,28
6,9
7,6
8,3
9,3


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [31]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,baths,mean_sale_price
0,NaN,1.739026
1,1.00,0.987656
2,1.10,1.420000
3,1.25,1.600000
4,1.50,1.223378
...,...,...
16,6.50,16.000000
17,7.00,0.999000
18,7.50,5.530000
19,8.00,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [34]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    WHERE baths = "1"
    GROUP BY baths
    
;
''', con = db)

,baths,mean_sale_price
0,1.0,0.987656


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [45]:
pd.io.sql.read_sql(
'''

SELECT a.id AS id_a, b.id AS id_b, a.address AS address_a, b.address AS address_b
    FROM properties AS a
    JOIN properties AS b ON b.latitude = a.latitude AND b.longitude = a.longitude
    WHERE a.address < b.address
;
''', con = db)

,id_a,id_b,address_a,address_b
0,52266124,53153575,"88 King St APT 317, San Francisco, CA","88 King St APT 419, San Francisco, CA"
1,119685526,119685188,"1902 Filbert St, San Francisco, CA","1902A Filbert St, San Francisco, CA"
2,79846666,79844263,"555 4th St UNIT 513, San Francisco, CA","555 4th St, San Francisco, CA"
3,61288494,61325457,"400 Alemany Blvd APT 12, San Francisco, CA","400 Alemany Blvd APT 15, San Francisco, CA"
4,89239837,89238065,"338 Spear St UNIT 10B, San Francisco, CA","338 Spear St UNIT 5J, San Francisco, CA"
...,...,...,...,...
102,115828655,115828679,"480 Mission Bay Blvd N UNIT 614, San Francisco...","480 Mission Bay Blvd N UNIT 809, San Francisco..."
103,15067890,15067896,"1250 Jones St APT 503, San Francisco, CA","1250 Jones St APT 802, San Francisco, CA"
104,111931130,111930969,"860 Waller St APT 1, San Francisco, CA","860 Waller St APT 2, San Francisco, CA"
105,60825591,61288454,"77 Dow Pl APT 500, San Francisco, CA","77 Dow Pl APT 507, San Francisco, CA"


In [69]:
pd.io.sql.read_sql(
'''

SELECT (latitude || ' ' || longitude) AS location, address
    FROM properties
    GROUP BY location
    HAVING COUNT(location)>1
    ORDER BY location
;
''', con = db)

,location,address
0,37710480 -122467645,"8400 Oceanview Ter APT 218, San Francisco, CA"
1,37725337 -122399307,"5800 3rd St UNIT 1401, San Francisco, CA"
2,37733174 -122414551,"400 Alemany Blvd APT 15, San Francisco, CA"
3,37742141 -122435111,"1915 Diamond St, San Francisco, CA"
4,37758487 -122412814,"2412 Harrison St APT 104, San Francisco, CA"
...,...,...
53,37798792 -122416797,"1070 Green St APT 1402, San Francisco, CA"
54,37798831 -122431053,"1902A Filbert St, San Francisco, CA"
55,37800573 -122424849,"2701 Van Ness Ave APT 207, San Francisco, CA"
56,37804240 -122405509,"111 Chestnut St APT 403, San Francisco, CA"
